In [ ]:
                                    # !pip install -U sagemaker

In [ ]:
from datetime import datetime

In [1]:
import sagemaker
sagemaker.__version__

'2.45.0'

In [6]:
from sagemaker.tensorflow import TensorFlow

In [7]:
from sagemaker import get_execution_role
role = get_execution_role()

In [2]:
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

In [43]:
data_path = f's3://{bucket}/image_classification/cats_and_dogs_small'

In [45]:
!aws s3 sync cats_and_dogs_small/ $data_path/

In [46]:
!aws s3 ls $data_path/

                           PRE test/
                           PRE train/
                           PRE validation/


In [47]:
train_data_path = f'{data_path}/train'
test_data_path = f'{data_path}/test'
validation_data_path = f'{data_path}/validation'

In [140]:
def get_training_info():
    timestamp = str(datetime.now())

    for symbol in [' ', ':', '.']:
        timestamp = timestamp.replace(symbol, '-')

    job_name = f'cats-dogs-{timestamp}'
    return job_name

In [52]:
train_data = sagemaker.inputs.TrainingInput(
    s3_data=train_data_path,
    distribution='FullyReplicated',
    content_type='application/x-image',
    s3_data_type='S3Prefix'
)

validation_data = sagemaker.inputs.TrainingInput(
    s3_data=validation_data_path,
    distribution='FullyReplicated',
    content_type='application/x-image',
    s3_data_type='S3Prefix'
)

test_data = sagemaker.inputs.TrainingInput(
    s3_data=test_data_path,
    distribution='FullyReplicated',
    content_type='application/x-image',
    s3_data_type='S3Prefix'
)

In [101]:
inputs = {
    'train': train_data_path,
    'test': test_data_path,
    'validation': validation_data_path,
}

# inputs = {
#     'train': train_data,
#     'test': test_data,
#     'validation': validation_data,
# }

In [139]:
model_output

's3://sagemaker-us-east-1-448873806592/image_classification/cats_and_dogs_small/output/cats-dogs-2021-06-14-06-27-27-836229'

In [164]:
job_name = get_training_info()

estimator = TensorFlow(
    source_dir='src/',
    entry_point="model_train.py",
    role=role,
    instance_count=1,
#     instance_type="ml.p3.2xlarge",
    instance_type="ml.m5.large",
#     instance_type='local',
    framework_version="2.4.1",
    py_version="py37",
    model_dir=f'{data_path}/output/{job_name}/',
    output_path=f'{data_path}/output/'
)

estimator.fit(inputs=inputs, job_name=job_name)

2021-06-14 07:11:00 Starting - Starting the training job...
2021-06-14 07:11:26 Starting - Launching requested ML instancesProfilerReport-1623654660: InProgress
......
2021-06-14 07:12:27 Starting - Preparing the instances for training......
2021-06-14 07:13:27 Downloading - Downloading input data...
2021-06-14 07:13:52 Training - Downloading the training image..2021-06-14 07:14:08.501038: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-06-14 07:14:08.506179: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-06-14 07:14:08.624164: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-06-14 07:14:12,686 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2021-06-14 07:14:12,694 sagemaker-training-

In [142]:
output_dir = 's3://sagemaker-us-east-1-448873806592/image_classification/cats_and_dogs_small/output'

In [168]:
!aws s3 ls $output_dir/cats-dogs-2021-06-14-07-11-00-382353-2021-06-14-07-11-00-458/output/

2021-06-14 07:14:18        139 model.tar.gz
2021-06-14 07:14:18        142 output.tar.gz


In [118]:
!aws s3 cp s3://sagemaker-us-east-1-448873806592/cats-dogs-2021-06-14-06-07-33-546568/output/

download: s3://sagemaker-us-east-1-448873806592/cats-dogs-2021-06-14-06-07-33-546568/output/output.tar.gz to ./output.tar.gz


In [157]:
!tar -xvzf model.tar.gz

file_model.txt


In [158]:
ls *txt

file_model.txt  file_output_data.txt


In [110]:
!aws s3 ls s3://sagemaker-us-east-1-448873806592/cats-dogs-2021-06-14-06-07-33-546568/profiler-output/system/incremental/2021061406/1623651000.algo-1.json

2021-06-14 06:11:23     113027 1623651000.algo-1.json


In [115]:
!aws s3 ls s3://sagemaker-us-east-1-448873806592/cats-dogs-2021-06-14-06-07-33-546568/rule-output/ProfilerReport-1623650853/profiler-output/profiler-reports/CPUBottleneck.json

2021-06-14 06:11:50        198 CPUBottleneck.json


In [117]:
!aws s3 ls s3://sagemaker-us-east-1-448873806592/cats-dogs-2021-06-14-06-07-33-546568/source/sourcedir.tar.gz

2021-06-14 06:07:35        675 sourcedir.tar.gz


In [ ]:
parser.add_argument('--model_dir', type=str)
parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
parser.add_argument('--test', type=str, default=os.environ.get('SM_CHANNEL_TEST'))

In [ ]:

sign_estimator.fit(input_data)

In [ ]:
def model(X_train, y_train, X_val, y_val):    
    model = models.Sequential()
    # Conv 32x32x1 => 28x28x6.
    model.add(layers.Conv2D(filters = 6, kernel_size = (5, 5), strides=(1, 1), padding='valid',
                            activation='relu', data_format = 'channels_last', input_shape = (32, 32, 1)))
    # Maxpool 28x28x6 => 14x14x6
    model.add(layers.MaxPooling2D((2, 2)))
    # Conv 14x14x6 => 10x10x16
    model.add(layers.Conv2D(16, (5, 5), activation='relu'))
    # Maxpool 10x10x16 => 5x5x16
    model.add(layers.MaxPooling2D((2, 2)))
    # Flatten 5x5x16 => 400
    model.add(layers.Flatten())
    # Fully connected 400 => 120
    model.add(layers.Dense(120, activation='relu'))
    # Fully connected 120 => 84
    model.add(layers.Dense(84, activation='relu'))
    # Dropout
    model.add(layers.Dropout(0.2))
    # Fully connected, output layer 84 => 43
    model.add(layers.Dense(43, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_val, y_val))

    return model


if __name__ == "__main__":
    args, unknown = _parse_args()

    train_data, train_labels = _load_training_data(args.train)
    eval_data, eval_labels = _load_validation_data(args.train)

    mdl = model(train_data, train_labels, eval_data, eval_labels)

    if args.current_host == args.hosts[0]:
        # save model to an S3 directory with version number '00000001'
        mdl.save(os.path.join(args.sm_model_dir, '000000001'), 'my_model.h5')